<a href="https://colab.research.google.com/github/SKam23/10315-Final-Project/blob/main/10315_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 10-315 ML Final Project
Created by Shaheer Aslam, Steven Kam, and Sajan Shah.

< explanation of project here >

## Library Imports

In [1]:
import importlib
import torch 
import torchvision.transforms as transforms
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
import torchvision.transforms.functional as TF
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor
from PIL import Image
from torchvision.utils import make_grid
from IPython.display import display

if importlib.util.find_spec("kaggle") is None:
    !pip install -q kaggle

# CelebA Dataset Processing

In [2]:
from google.colab import drive
drive.mount('/content/drive')

os.chdir('/content/drive/MyDrive/Colab Notebooks/final project/')


Mounted at /content/drive


In [3]:
# load images into a dictionary for easier access
images = {}
for filename in os.listdir('data/selected_images/'):
    if filename.endswith('.jpg'):
        image = Image.open(os.path.join('data/selected_images/', filename))
        images[filename] = image


In [4]:
print(images)

{'100926.jpg': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=178x218 at 0x7F1C3CA0B940>, '103164.jpg': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=178x218 at 0x7F1B773C2AA0>, '102797.jpg': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=178x218 at 0x7F1B6DB1CF40>, '102508.jpg': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=178x218 at 0x7F1B6DB1CF10>, '102267.jpg': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=178x218 at 0x7F1B6DB1CE20>, '102247.jpg': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=178x218 at 0x7F1B773C2A40>, '102958.jpg': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=178x218 at 0x7F1B6DB1CFA0>, '102767.jpg': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=178x218 at 0x7F1B6DB1CEB0>, '103001.jpg': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=178x218 at 0x7F1B6DB1CEE0>, '103866.jpg': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=178x218 at 0x7F1B6DB1CE80>, '103132.jpg': <PIL.JpegImageP

In [5]:
# list_eval_partition.csv: Recommended partitioning of images into training, validation, testing sets. Images 1-162770 are training, 162771-182637 are validation, 182638-202599 are testing
# list_bbox_celeba.csv: Bounding box information for each image. "x_1" and "y_1" represent the upper left point coordinate of bounding box. "width" and "height" represent the width and height of bounding box
# list_landmarks_align_celeba.csv: Image landmarks and their respective coordinates. There are 5 landmarks: left eye, right eye, nose, left mouth, right mouth
# list_attr_celeba.csv: Attribute labels for each image. There are 40 attributes. "1" represents positive while "-1" represents negative
partition_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/final project/data/list_eval_partition2000Smile.csv')
bbox_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/final project/data/list_bbox_celeba2000Smile.csv')
landmarks_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/final project/data/list_landmarks_align_celeba2000Smile.csv')
attr_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/final project/data/list_attr_celeba2000Smile.csv')



In [6]:
merged_df = partition_df.merge(bbox_df, on='image_id').merge(landmarks_df, on='image_id').merge(attr_df, on='image_id')
train_df = merged_df[merged_df['partition'] == 0]
val_df = merged_df[merged_df['partition'] == 1]
test_df = merged_df[merged_df['partition'] == 2]


In [7]:
train_df

,image_id,partition,x_1,y_1,width,height,lefteye_x,lefteye_y,righteye_x,righteye_y,...,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young
0,000219.jpg,0,291,78,122,169,68,110,107,113,...,-1,-1,-1,1,-1,-1,-1,-1,-1,1
1,000269.jpg,0,142,104,148,205,68,111,109,112,...,-1,1,-1,1,1,-1,1,-1,-1,1
2,000368.jpg,0,120,49,96,133,69,111,108,112,...,-1,1,-1,-1,-1,-1,-1,-1,-1,1
3,000428.jpg,0,60,67,107,148,68,111,109,112,...,-1,1,-1,-1,-1,-1,-1,-1,1,1
4,000555.jpg,0,72,20,122,169,68,112,109,112,...,1,1,-1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1608,162387.jpg,0,87,94,166,230,69,110,107,112,...,-1,1,-1,1,1,-1,1,-1,-1,1
1609,162461.jpg,0,181,53,242,335,69,112,108,111,...,-1,-1,1,-1,-1,-1,1,-1,-1,1
1610,162624.jpg,0,188,159,203,281,68,112,109,112,...,-1,1,-1,1,-1,-1,1,-1,-1,1
1611,162656.jpg,0,189,114,361,500,70,111,107,111,...,-1,-1,-1,-1,1,-1,1,-1,-1,1


In [8]:
class CelebADataset(Dataset):
    def __init__(self, df, images):
        self.df = df
        self.images = images
        self.transform = transforms.Compose([transforms.Resize((178, 178)),transforms.ToTensor(),])
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image_id = row['image_id']
        image = self.images[image_id]
        image = self.transform(image)
        is_smiling = row["Smiling"] == 1
        label = 1 if is_smiling else 0
        return image,label
        # return image, (label, image_id)

In [9]:
train_dataset = CelebADataset(train_df, images)
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)

val_dataset = CelebADataset(val_df, images)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=False)

test_dataset = CelebADataset(test_df, images)
test_dataloader = DataLoader(test_dataset, batch_size=64)

In [10]:
row = train_dataset.df.iloc[0]
row["Smiling"]

-1

In [11]:
import random

def get_random_images(dataset):
    smile_images = []
    not_smile_images = []
    
    for i in range(len(dataset)):
        image, label = train_dataset.__getitem__(i)
        if label[0] == 1:
            smile_images.append(dataset.images[label[1]])
        else:
            not_smile_images.append(dataset.images[label[1]])
    
    print(len(smile_images))
    print(len(not_smile_images))
    smile_image = random.choice(smile_images)
    not_smile_image = random.choice(not_smile_images)

    
    return smile_image, not_smile_image


In [12]:
# young_image, not_young_image = get_random_images(train_dataset)

# # Create a figure and set title
# fig = plt.figure(figsize=(8, 4))
# fig.suptitle('Smiling vs Not Smiling Images', fontsize=16)

# # Add young image to subplot 1
# ax1 = fig.add_subplot(1, 2, 1)
# ax1.imshow(young_image)
# ax1.set_title('Smiling Image')

# # Add not young image to subplot 2
# ax2 = fig.add_subplot(1, 2, 2)
# ax2.imshow(not_young_image)
# ax2.set_title('Not Smiling Image')

# plt.show()

# Convolutional Neural Network
< explanation >

In [13]:
# # We create a PyTorch CNN from scratch, using two convolutional layers and one fully connected layer (for now).
# class CNN(nn.Module):
#   def __init__(self):
#     # supering nn.Module
#     super(CNN, self).__init__()

#     # creating first convolutional layer
#     self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
#     self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

#     # creating second convolutional layer
#     self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
#     self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

#     # creating third convolutional layer
#     self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
#     self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

#     # creating fully connected layers
#     self.fc1 = nn.Linear(64 * 8 * 8, 128)
#     self.fc2 = nn.Linear(128, 128)
#     self.fc3 = nn.Linear(128, 40)
  
#   # feed forward portion of CNN
#   def forward(self, x):
#     # first, second and third convolutional layer
#     x = self.pool1(torch.relu(self.conv1(x)))
#     x = self.pool2(torch.relu(self.conv2(x)))
#     x = self.pool3(torch.relu(self.conv3(x)))

#     # passing fully connected layers through relu activation layers (except last one)
#     x = x.view(-1, 64 * 8 * 8)
#     x = torch.relu(self.fc1(x))
#     x = torch.relu(self.fc2(x))
#     x = self.fc3(x)
#     return x

In [14]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 44 * 44, 2)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        return x


In [15]:
import torch.nn.functional as F

def train(model, train_dataloader, val_dataloader, num_epochs):
    # define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters())

    train_losses, val_losses = [], []
    for epoch in range(num_epochs):
        # training loop
        model.train()
        train_loss = 0.0
        for inputs, targets in train_dataloader:
          print(inputs.size())
          optimizer.zero_grad()
          outputs = model(inputs)
          targets_onehot = F.one_hot(targets, num_classes=2).float()  # convert targets to one-hot format
          loss = criterion(outputs, targets_onehot)
          loss.backward()
          optimizer.step()
          train_loss += loss.item() * inputs.size(0)
        train_loss /= len(train_dataloader.dataset)
        train_losses.append(train_loss)

        # validation loop
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, targets in val_dataloader:
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                val_loss += loss.item() * inputs.size(0)
            val_loss /= len(val_dataloader.dataset)
            val_losses.append(val_loss)

        # print progress
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

    return train_losses, val_losses


In [16]:
train(CNN(), train_dataloader,val_dataloader,10)

torch.Size([64, 3, 178, 178])
torch.Size([64, 3, 178, 178])
torch.Size([64, 3, 178, 178])
torch.Size([64, 3, 178, 178])
torch.Size([64, 3, 178, 178])
torch.Size([64, 3, 178, 178])
torch.Size([64, 3, 178, 178])
torch.Size([64, 3, 178, 178])
torch.Size([64, 3, 178, 178])
torch.Size([64, 3, 178, 178])
torch.Size([64, 3, 178, 178])
torch.Size([64, 3, 178, 178])
torch.Size([64, 3, 178, 178])
torch.Size([64, 3, 178, 178])
torch.Size([64, 3, 178, 178])
torch.Size([64, 3, 178, 178])
torch.Size([64, 3, 178, 178])
torch.Size([64, 3, 178, 178])
torch.Size([64, 3, 178, 178])
torch.Size([64, 3, 178, 178])
torch.Size([64, 3, 178, 178])
torch.Size([64, 3, 178, 178])
torch.Size([64, 3, 178, 178])
torch.Size([64, 3, 178, 178])
torch.Size([64, 3, 178, 178])
torch.Size([13, 3, 178, 178])
Epoch [1/10], Train Loss: 0.9604, Val Loss: 0.6875
torch.Size([64, 3, 178, 178])
torch.Size([64, 3, 178, 178])
torch.Size([64, 3, 178, 178])
torch.Size([64, 3, 178, 178])
torch.Size([64, 3, 178, 178])
torch.Size([64, 3, 

([0.9603935361869278,
  0.6741130437011388,
  0.5701108472323609,
  0.4268215554365074,
  0.34964972560773777,
  0.2783473944412723,
  0.24003142441789777,
  0.2622482941727866,
  0.19120964974220392,
  0.15366089737108607],
 [0.6874519923458928,
  0.6842415928840637,
  0.5823395615038665,
  0.47138457972070447,
  0.40569958349932794,
  0.4157820305098658,
  0.38074611192164215,
  0.4077276483826015,
  0.34890422484149103,
  0.3656429104183031])

In [17]:
# def train(model, dataloader, criterion, optimizer, device):
#     model.train()
#     running_loss = 0.0
#     for i, (inputs, labels) in enumerate(dataloader):
#         inputs, labels = inputs.to(device), labels.to(device)
#         optimizer.zero_grad()
#         outputs = model(inputs)
#         loss = criterion(outputs, labels[:, 0])
#         loss.backward()
#         optimizer.step()
#         running_loss += loss.item()
#     return running_loss / len(dataloader)


# def test(model, dataloader, criterion, device):
#     model.eval()
#     running_loss = 0.0
#     correct = 0
#     total = 0
#     with torch.no_grad():
#         for i, (inputs, labels) in enumerate(dataloader):
#             inputs, labels = inputs.to(device), labels.to(device)
#             outputs = model(inputs)
#             loss = criterion(outputs, labels[:, 0])
#             running_loss += loss.item()
#             _, predicted = torch.max(outputs.data, 1)
#             total += labels.size(0)
#             correct += (predicted == labels[:, 0]).sum().item()
#     return running_loss / len(dataloader), correct / total


# Training and Testing

In [18]:
# # creating model
# model = CNN()
# criterion = nn.BCEWithLogitsLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# model.to(device)

In [19]:
test(model, test_dataloader)

NameError: ignored